In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_334704/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_49_9,0.999888,0.711082,0.731052,0.999360,0.000057,0.171514,0.188403,0.000175,0.002105,0.007567,1.000090,0.007832,149.535908,240.613737,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_49_8,0.999886,0.711184,0.730916,0.999352,0.000058,0.171454,0.188499,0.000177,0.002104,0.007629,1.000092,0.007896,149.503426,240.581256,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_49_7,0.999884,0.711291,0.730771,0.999341,0.000059,0.171390,0.188600,0.000180,0.002104,0.007697,1.000093,0.007967,149.467794,240.545624,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_49_6,0.999882,0.711395,0.730620,0.999329,0.000060,0.171328,0.188706,0.000184,0.002104,0.007771,1.000095,0.008044,149.429499,240.507329,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_49_5,0.999880,0.711511,0.730489,0.999318,0.000061,0.171259,0.188798,0.000186,0.002104,0.007838,1.000097,0.008113,149.395275,240.473104,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.923874,0.687698,0.664101,0.908033,0.038926,0.185396,0.235304,0.035215,0.114693,0.197297,1.061324,0.204221,136.492187,227.570017,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.922324,0.687091,0.663269,0.906144,0.039718,0.185757,0.235887,0.035938,0.115835,0.199295,1.062572,0.206290,136.451877,227.529707,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.920742,0.686461,0.662414,0.904213,0.040528,0.186130,0.236486,0.036678,0.116989,0.201314,1.063847,0.208380,136.411549,227.489378,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.917480,0.685139,0.660634,0.900224,0.042196,0.186915,0.237733,0.038205,0.119331,0.205416,1.066475,0.212626,136.330874,227.408704,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
